# Part 1

## Premise
- a cephalopod asks us to help with its math homework.
- the homework has three rows of numbers with an operation on the fourth row. Apply that operation to the respective column values, then sum all of those results for a 'grand total'

## thoughts
- this is an excel spreadsheet task, or an opportunity to use Pandas
- make a dataframe, calculate results, append to dataframe for viewing, sum new row of results

In [1]:
import pandas as pd
import numpy as np

In [2]:
# df = pd.read_csv("input.txt", delim_whitespace=True, header=None)
df = pd.read_csv("input.txt", sep="\\s+", header=None)

columnResult = []
for i in df:
    arguments = df[i][0:4]
    if df[i][4] == '+':
        result = np.sum(arguments.astype(int))
        columnResult.append(result)
    if df[i][4] == '*':
        result = np.prod(arguments.astype(int))
        columnResult.append(result)

df.loc[5] = columnResult
grandTotal = np.sum(df.iloc[5])
print(f"Grand total of hw: {grandTotal}")
df

Grand total of hw: 6171290547579


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,44,493,13,675,94,42,551,95,5,49,...,285,52,168,23,6,58,84,39,1,2261
1,57,584,34,874,216,1,613,57,4,23,...,132,245,499,23,524,79,78,41,9,9718
2,86,983,824,132,9554,9,629,867,47,78,...,319,543,5958,556,448,118,93,57,99,261
3,35,859,233,18,7852,1,8,773,75,56,...,7,389,7127,267,664,142,465,59,36,87
4,*,*,+,*,+,+,*,+,*,+,...,*,+,+,+,*,+,+,+,*,+
5,7549080,243112029064,1104,1401721200,17716,53,1699623416,1792,70500,206,...,84005460,1229,13752,869,935251968,397,720,196,32076,12327


# Part 2

## Change
- woops we did 'human numbers', this is cephalopod math... We have to read numbers vertically top-down going right to left, so
<pre>
    123                                                     123
     35     reads as     356, 23, 1         whereas         35      reads as     3, 25, 136
      6                                                     6
</pre>
- skimming the input file, the columns are not consistently left or right aligned, this causes issues ^
  - we can't use positional values (1,10,100) due to inconsistent alignment as shown above
  - when values are read into dataframe, they are stripped of whitespace, losing necessary information
- might have to ditch panda.read_csv and write input process

In [3]:
# here we get the tuple that contains the slice values that identify every value in its line

IndicesTable = [] # 2d array containing slice tuples that retrieve corresponding value 
lines = []
# we can use 
infile = open("input.txt", 'r')
for line in infile:
    thisRow = []
    lines.append(line)
    firstWhitespace = True
    temp = []
    for i, char in enumerate(line):
        if (48 <= ord(char) <= 57) or ord(char) == 42 or ord(char)==43: # if numeric or * or +
            firstWhitespace = True
            temp.append(i)
            
        else:
            if firstWhitespace:
                thisRow.append((min(temp),max(temp))) # save start and end indices on first space
                temp = []
                firstWhitespace = False
            else:
                continue 
        
    IndicesTable.append(thisRow)


In [4]:

columnResults = []
for column in range(1000):
    bounds = [IndicesTable[0][column], IndicesTable[1][column], IndicesTable[2][column], IndicesTable[3][column], IndicesTable[4][column]]
    maxIdx = max([bounds[i][1] for i in range(4)])
    minIdx = min([bounds[i][0] for i in range(4)])
    arguments = []

    # check if left aligned, i.e. all start bounds equal
    if IndicesTable[0][column][0] == IndicesTable[1][column][0] == IndicesTable[2][column][0] == IndicesTable[3][column][0]:
        for idx in range(maxIdx, minIdx - 1, -1): # from right to left
            cephNumDigits = [lines[rowNum][idx] for rowNum in range(4)] # read digits from top to bottom
            cephNumsStr = "" # convert digits array to int
            for digit in cephNumDigits:
                cephNumsStr += f"{digit}"
            arguments.append(int(cephNumsStr))
        

    # check if right-aligned, i.e. all end bounds equal [elif otherwise full columns count as both, see first input column for example]
    elif IndicesTable[0][column][1] == IndicesTable[1][column][1] == IndicesTable[2][column][1] == IndicesTable[3][column][1]:
        for idx in range(maxIdx, minIdx - 1, -1): # from right to left
            cephNumDigits = [lines[rowNum][idx] for rowNum in range(4)] # digits are read from top to bottom
            cephNumsStr = "" # convert digits array to int
            for digit in cephNumDigits:
                cephNumsStr += f"{digit}"
            arguments.append(int(cephNumsStr))
        
    # find operator and apply to arguments
    operatorIdx = IndicesTable[4][column][0]
    operator = lines[4][operatorIdx]
    result = 0
    match operator:
        case '+':
            result = np.sum(arguments)
        case '*':
            result = np.prod(arguments)
    # print(f"arguments: {arguments}    operator: {operator}    result: {result}")
    columnResults.append(result)

grandTotal = np.sum(columnResults)
print(f"Grand total of cephalopod homework: {grandTotal}")

Grand total of cephalopod homework: 8811937976367


# Done!
   
## Part 1
- answer: 6171290547579
- It took a second to remember the DataFrame methods but worked out smoothly.

## Part 2
- answer: 8811937976367
- learned about using `<pre> content </pre>` to preserve whitespace and raw formatting in markdown, borrowed from html
- The manual file input was not as bad as I expected, but certainly did take a bit of checking along the way. Thankfully, this is a notebook.